In [29]:
from pandas import read_csv
import matplotlib.pyplot as plt
import numpy
import math
import seaborn as sns
from datetime import datetime
import tzlocal
from sklearn.preprocessing import MinMaxScaler

# Model
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Validation tool
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error

In [19]:
dataset = read_csv('sample.csv')

In [20]:
print dataset

                  TRIP_ID CALL_TYPE  ORIGIN_CALL  ORIGIN_STAND   TAXI_ID  \
0     1372641403620000424         C          NaN           NaN  20000424   
1     1372647683620000492         B          NaN          58.0  20000492   
2     1372653015620000128         A      47490.0           NaN  20000128   
3     1372660434620000128         B          NaN          11.0  20000128   
4     1372669161620000195         A      60725.0           NaN  20000195   
5     1372672171620000684         B          NaN           NaN  20000684   
6     1372673912620000142         C          NaN           NaN  20000142   
7     1372674990620000518         B          NaN          36.0  20000518   
8     1372677386620000314         A      13462.0           NaN  20000314   
9     1372682122620000198         B          NaN          36.0  20000198   
10    1372682170620000603         C          NaN           NaN  20000603   
11    1372685332620000602         C          NaN           NaN  20000602   
12    137268

In [22]:
print dataset.TIMESTAMP

0       1372641403
1       1372647683
2       1372653015
3       1372660434
4       1372669161
5       1372672171
6       1372673912
7       1372674990
8       1372677386
9       1372682122
10      1372682170
11      1372685332
12      1372688258
13      1372689274
14      1372689330
15      1372690129
16      1372701055
17      1372702178
18      1372705518
19      1372706118
20      1372706662
21      1372709509
22      1372712491
23      1372713649
24      1372734256
25      1372738530
26      1372748748
27      1372757936
28      1372759112
29      1372759806
           ...    
9970    1404056912
9971    1404059167
9972    1404064534
9973    1404068027
9974    1404072214
9975    1404077040
9976    1404077988
9977    1404084952
9978    1404085550
9979    1404087847
9980    1404089696
9981    1404091849
9982    1404114261
9983    1404115308
9984    1404116552
9985    1404117050
9986    1404118110
9987    1404118917
9988    1404125357
9989    1404129764
9990    1404139439
9991    1404

In [52]:
unix_timestamp = dataset.TIMESTAMP.astype('float')

In [74]:
from sklearn import preprocessing

dataset_poly = read_csv('sample.csv', usecols=['POLYLINE'])
poly_values  = dataset_poly.values
print dataset_poly
print poly_values

                                               POLYLINE
0     [[-8.609391,41.1399],[-8.609481,41.140197],[-8...
1     [[-8.604864,41.161149],[-8.605053,41.160789],[...
2     [[-8.609256,41.179401],[-8.609256,41.179392],[...
3     [[-8.613333,41.166864],[-8.613711,41.166801],[...
4     [[-8.575434,41.156262],[-8.575461,41.156271],[...
5     [[-8.600193,41.182713],[-8.599905,41.182704],[...
6     [[-8.669907,41.148522],[-8.669916,41.148504],[...
7     [[-8.64954,41.154282],[-8.650269,41.154165],[-...
8     [[-8.626986,41.158746],[-8.626941,41.158692],[...
9     [[-8.649261,41.154183],[-8.649981,41.154129],[...
10    [[-8.665092,41.170995],[-8.66502,41.171022],[-...
11    [[-8.614107,41.162058],[-8.613558,41.161977],[...
12    [[-8.612379,41.145894],[-8.612514,41.14602],[-...
13    [[-8.614161,41.141106],[-8.615646,41.140503],[...
14    [[-8.604261,41.160618],[-8.604153,41.160897],[...
15    [[-8.652825,41.156838],[-8.652888,41.155758],[...
16    [[-8.606934,41.150205],[-8.607105,41.15016

In [111]:
from sklearn import preprocessing
taxi_id = dataset.TAXI_ID
le = preprocessing.LabelEncoder()
le.fit(taxi_id)
label_taxi_id = le.transform(taxi_id)
print label_taxi_id

[266 307  91 ..., 414 222  64]


In [113]:
max_id = max(label_taxi_id)
norm_taxi_id = label_taxi_id.astype('float32') / max_id
print norm_taxi_id

[ 0.60869563  0.70251715  0.20823799 ...,  0.94736844  0.50800914
  0.14645308]


In [168]:
unix_timestamp = dataset.TIMESTAMP.astype('float32')

#print("index, element, utc_time")
for index, element in enumerate(unix_timestamp):
    utc_time = datetime.utcfromtimestamp(element)
    #print("%s, %s, %s" % (index, element, utc_time))
    #unix_timestamp[index] = utc_time
    unix_timestamp[index] = int(str(utc_time)[11:13])
    
print unix_timestamp

0        1.0
1        3.0
2        4.0
3        6.0
4        8.0
5        9.0
6       10.0
7       10.0
8       11.0
9       12.0
10      12.0
11      13.0
12      14.0
13      14.0
14      14.0
15      14.0
16      17.0
17      18.0
18      19.0
19      19.0
20      19.0
21      20.0
22      21.0
23      21.0
24       3.0
25       4.0
26       7.0
27       9.0
28       9.0
29      10.0
        ... 
9970    15.0
9971    16.0
9972    17.0
9973    18.0
9974    20.0
9975    21.0
9976    21.0
9977    23.0
9978    23.0
9979     0.0
9980     0.0
9981     1.0
9982     7.0
9983     8.0
9984     8.0
9985     8.0
9986     8.0
9987     9.0
9988    10.0
9989    12.0
9990    14.0
9991    15.0
9992    15.0
9993    16.0
9994    16.0
9995    17.0
9996    18.0
9997    20.0
9998    21.0
9999    21.0
Name: TIMESTAMP, Length: 10000, dtype: float32


In [160]:
time = unix_timestamp[0].strftime("%Y-%m-%d %H:%M:%S(UTC)")
print time

2013-07-01 01:16:48(UTC)


In [161]:
print str(time)[11:13]

01


In [162]:
print str(time)[8:11]

01 
